# Spam or Ham Classification in Email Using DistilBERT Transformer

In [27]:
# import libarires
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import DistilBertTokenizerFast

from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments


In [12]:
df = messages = pd.read_csv('SMSSpamCollection.txt',sep='\t',
                            names=['label','message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
df.shape

(5572, 2)

In [15]:
df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [16]:
# independent feature
X = list(df['message'])

In [17]:
y = list(df['label'])

In [18]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])


In [20]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


### `Step to be followed :`
- Call the pretrained model 
- Call the tokenizer
- Convert encoding into Dataset objects


In [44]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

In [36]:
# train_encoding = tokenizer(X_train, truncation=True, padding=True)
# test_encoding = tokenizer(X_test, truncation=True, padding=True)


### Converting these encoding into Dataset objects

In [45]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encoding), y_test))

In [46]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.bool, name=None))>

In [51]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
import tensorflow as tf
from transformers import Trainer

# Define training arguments
training_args = TFTrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,  # batch size for training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    # logging_dir="./logs",            # directory for storing logs
    logging_steps=10
)

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    
)

trainer.train()

AttributeError: 'NoneType' object has no attribute 'items'

In [41]:
# training_args = TFTrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=2,              # total number of training epochs
#     per_device_train_batch_size=4,  # batch size per device during training
#     per_device_eval_batch_size=8,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,)
# Define distribution strategy
# Define distribution strategy
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Load pre-trained model
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

    # Define training arguments
    training_args = TFTrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
    )

    # Initialize Trainer
    trainer = TFTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset
    )

    # Train the model
    trainer.train()
        

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


AttributeError: 'NoneType' object has no attribute 'items'

In [34]:
# with training_args.strategy.scope():
#     model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# trainer = TFTrainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=test_dataset             # evaluation dataset
# )

# trainer.train()
